In [26]:
# Main file
# Use case: Pilot test, "integration test"

# === STEP 0: Imports
print("# === STEP 0: Imports: Starting ...")
from modules.dataset_generator.helpers.configuration_loader import ConfigurationLoader as DSConfigLoader
from modules.dataset_generator.factories.data_io_factory import DataIOFactory
from modules.dataset_generator.factories.feature_processor_factory import FeatureProcessorFactory
from modules.dataset_generator.factories.join_factory import JoinFactory
from modules.dataset_generator.factories.strategy_factory import StrategyFactory
from modules.dataset_generator.dataset_generator import DatasetGenerator
from modules.processor.helpers.configuration_loader import ConfigurationLoader as PConfigLoader
from modules.processor.factories.split_strategy_factory import SplitStrategyFactory
from modules.processor.processor import Processor
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
print("# === STEP 0: Imports: Complete")

# === STEP 0: Imports: Starting ...
# === STEP 0: Imports: Complete


In [27]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...")
## === DATASET GEN
yaml_path = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v0.yaml'
ds_configuration_loader = DSConfigLoader()
data_factory, feature_processor_factory, join_factory, strategy_factory = DataIOFactory(), FeatureProcessorFactory(), JoinFactory(), StrategyFactory()
## === PROCESSOR
p_configuration_loader = PConfigLoader()
split_strategy_factory = SplitStrategyFactory()
## === MODEL MANAGER
checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete")

# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...
# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete


In [5]:
# === STEP 2: DATASET GEN
print("# === STEP 2: DATASET GEN: Starting ...")
dataset_generator = DatasetGenerator(yaml_path, ds_configuration_loader, data_factory, feature_processor_factory, join_factory, strategy_factory)

# === STEP 2: DATASET GEN: Starting ...


In [ ]:
processed_dataset = dataset_generator.generate()
print("# === STEP 2: DATASET GEN: Complete")

In [ ]:
## Use to save processed dataset to disk ## 
import pickle
import os

# File path to save and load the processed dataset
processed_dataset_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/processed_dataset.pkl"

with open(processed_dataset_path, "wb") as f:
        pickle.dump(processed_dataset, f)
        print("# === STEP 2: DATASET GEN: Saved to file")

In [ ]:
## Use to load processed dataset if computed already ## 
import pickle
import os

print("# === STEP 2: DATASET GEN: Loading from saved file")
with open(processed_dataset_path, "rb") as f:
    processed_dataset = pickle.load(f)

In [ ]:
processed_dataset.features

In [29]:
# === STEP 3: PROCESSOR
print("# === STEP 3: PROCESSOR: Starting ...")
processor = Processor(yaml_path, p_configuration_loader, processed_dataset, split_strategy_factory)

# === STEP 3: PROCESSOR: Starting ...


In [30]:
train_dataset, validation_dataset, test_dataset = processor.generate()
print("# === STEP 3: PROCESSOR: Complete")

# === STEP 3: PROCESSOR: Complete


In [36]:
# === STEP 4: MODEL MANAGER
print("# === STEP 4: MODEL MANAGER: Starting ...")
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)
models = model_manager.create_models([yaml_path])

# === STEP 4: MODEL MANAGER: Starting ...


In [37]:
train_dataset.examples[0].features
len(train_dataset.examples[0].features) # should be 177

177

In [ ]:
accuracies = model_manager.train([models], [(train_dataset, validation_dataset)], save_after_training=True)
print("# === STEP 4: MODEL MANAGER: Complete")

In [ ]:
## Using test_dataset find model accuracy
## Can use model.predict but would need to calculate accuracy 

In [ ]:
print("# === STEP 5: Model Outcome & Evaluation: Starting ...")
import matplotlib.pyplot as plt
# Unpack and graph accuracies
for i, (train_accuracies, val_accuracies) in enumerate(accuracies):
    model_signature = models[i].get_training_config().model_signature

    # Prepare the x-axis (epochs)
    epochs = range(1, len(train_accuracies) + 1)

    # Create a plot
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color="blue")
    if val_accuracies is not None:
        plt.plot(epochs, val_accuracies, label="Validation Accuracy", color="orange")

    # Add labels, title, and legend
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(f"Training and Validation Accuracy - {model_signature}")
    plt.legend()

    plt.show()

    print(f"Accuracy plot saved for model: {model_signature}")
print("# === STEP 5: Model Outcome & Evaluation: Complete")